In [1]:
import local_models.local_models
import local_models.loggin
import local_models.TLS_models
import local_models.linear_projections
import local_models.utils
import local_models.algorithms
import local_models.visualization
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import scipy.stats
import logging
import ml_battery.log
import time
import os
from IPython.display import Markdown as md
import joblib
import tqdm
import functools
import itertools

logger = logging.getLogger(__name__)

reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')

def import_shit():
    import local_models.local_models
    import local_models.loggin
    import local_models.TLS_models
    import local_models.linear_projections
    import local_models.utils
    import local_models.algorithms
    import numpy as np
    import matplotlib.pyplot as plt
    import sklearn.linear_model
    import sklearn.cluster
    from importlib import reload
    from ml_battery.utils import cmap
    import matplotlib as mpl
    import sklearn.datasets
    import scipy.stats
    import logging
    import ml_battery.log
    import time
    import os
    from IPython.display import Markdown as md
    import joblib
    import itertools
    import functools

In [2]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [3]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [4]:
project_dir = "../data/moons_reverse_tls_smooth_01"
os.makedirs(project_dir, exist_ok=1)

In [5]:
n = 200
noise=.06
SEED = 1
np.random.seed(SEED)
KERNEL=local_models.local_models.GaussianKernel
noisy_moons, c = sklearn.datasets.make_moons(n_samples=n, noise=noise)

In [6]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(1))
linear_models.fit(noisy_moons)

In [17]:
def reverse_local_tls_shift(linear_models, data, bandwidth, iterations=1000, kernel=None):
    timelog = local_models.loggin.TimeLogger(logger=logger, how_often=1, total=iterations, tag="reverse_local_tls_shift")
    kernel = kernel(bandwidth)
    yield data
    for i in range(iterations):
        with timelog:
            linear_params_vecs, linear_params_mean = local_models.linear_projections.transformate_data(data, kernel, linear_models, r=kernel.support_radius())
            blah = local_models.utils.linear_project_pointwise_bases(data, linear_params_vecs, linear_params_mean)
            data = (data - blah)*0.01
            err_pts = np.any(np.isnan(linear_params_vecs), axis=(1,2))
            logger.info("linear odr undefined at {} pts".format(err_pts.sum()))
            good_pts = np.logical_not(err_pts)
            data, linear_params_vecs, linear_params_mean = data[good_pts], linear_params_vecs[good_pts], linear_params_mean[good_pts]
            yield data


In [18]:
def make_reverse_tlsline_iterprojections_animation(linear_models, data, c, pth, bandwidth, range_pct=0.1, grid_steps=100, iterations=100, kernel=local_models.local_models.TriCubeKernel):
    shifter_maker = lambda projections: reverse_local_tls_shift(linear_models, projections, bandwidth, iterations=iterations, kernel=kernel)
    local_models.visualization.iterprojections_animation(shifter_maker, data, c, pth, bandwidth, range_pct=range_pct, grid_steps=grid_steps)

In [ ]:
fig_dir_lines = os.path.join(project_dir, "reverse_iterprojections")
make_reverse_tlsline_iterprojections_animation(linear_models, noisy_moons, c, fig_dir_lines, 
    0.2, kernel=KERNEL
)